In [105]:
import pandas as pd
from i7_hot_reload.hot_reload import hot_reload_a_module_from_cwd


b_000_env = hot_reload_a_module_from_cwd('b_000_env')

from b_000_env import Directory
from b_000_env import FilePath
from b_000_env import Constants


d = Directory()
fp = FilePath()
c = Constants()

# READ RAW DATA
## APPS WIDE DATA - DATA FROM OTREE

In [60]:
df_apps_wide__raw = pd.read_csv(fp.all_apps_wide_csv_250301)
print(df_apps_wide__raw.shape)
df_apps_wide__raw.head()

(300, 296)


,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,EndAndPayment.1.player.payoff,EndAndPayment.1.player.PairwisePaymentPage_timestamp,EndAndPayment.1.player.NodewisePaymentPage_timestamp,EndAndPayment.1.player.ThanksPage_timestamp,EndAndPayment.1.player.lottery,EndAndPayment.1.player.final_payment,EndAndPayment.1.player.recon_choice,EndAndPayment.1.player.comment,EndAndPayment.1.group.id_in_subsession,EndAndPayment.1.subsession.round_number
0,1,xjulaqnt,NaN,0,1,33,Introduction,Introduction,2025-02-11 15:47:47.700346,1,...,0.0,0,0,0,NaN,NaN,NaN,NaN,1,1
1,2,vc2z1rqk,NaN,0,33,33,EndAndPayment,ThanksPage,2025-02-11 16:05:37.167970,1,...,0.0,0,0,1739290857611,NaN,NaN,NaN,NaN,1,1
2,3,kb6n7gdo,NaN,0,33,33,EndAndPayment,ThanksPage,2025-02-11 18:21:57.987464,1,...,0.0,0,0,1739299111412,NaN,NaN,NaN,NaN,1,1
3,4,4ehzz2gc,NaN,0,33,33,EndAndPayment,ThanksPage,2025-02-11 18:32:54.168255,1,...,0.0,0,0,1739299885508,NaN,NaN,NaN,NaN,1,1
4,5,t6jmltim,NaN,0,33,33,EndAndPayment,ThanksPage,2025-02-11 18:49:17.748246,1,...,0.0,0,0,1739301525273,NaN,NaN,NaN,NaN,1,1


## FINAL PAYMENTS DATA - DATA FROM MTURK FINAL PAYMENTS EXCEL FILE

In [61]:
df_payments__raw = pd.read_excel(fp.final_payments)
print(df_payments__raw.shape)
df_payments__raw.head()

(48, 5)


,workerId,Track,Base,Bonus,Total
0,A3CMWYLWMENHLZ,CSI,1.6,1.6,3.2
1,A2FB5UQTR3E0NX,CSI,1.6,5.6,7.2
2,A3K1JTEB70E4FV,CSI,1.6,5.6,7.2
3,A1O4W7F05T6HY0,CSI,1.6,0,1.6
4,A37SLQRKOALNV6,CSI,1.6,5.6,7.2


# DROP NO MTRUK ID ROWS FROM BOTH DATA

In [62]:
# Keep only those with MTurk ID in apps_wide data
df_apps = df_apps_wide__raw.dropna(subset = [c.mturk_id])
print(df_apps.shape)

(47, 296)


In [63]:
# keep only rows with workerId value [dropping the total row]
df_pay = df_payments__raw.dropna(subset = [c.workerId])
print(df_pay.shape)

(46, 5)


# REMAINING SET OF IDS ARE NOT THE SAME - 1 MORE IN APPS DATA

# CHECK ON THAT 1 DIFFERENCE
find that one

In [64]:
ids_from_df_apps = set(df_apps[c.mturk_id])
print(len(ids_from_df_apps))

ids_from_df_pay = set(df_pay[c.workerId])
print(len(ids_from_df_pay))

sym_diff = ids_from_df_apps.symmetric_difference(ids_from_df_pay)
print(len(sym_diff))
print(sym_diff)

47
46
1
{'ALT6TQMZJKSFQ'}


## MANUAL CHECK
I manually checked whether this workerId (ALT6TQMZJKSFQ) is present in the

    1. bonus_amounts.xlsx or
    2. bonus_payments_to_incentivized_February_18th_2025.xlsx
    3. AssignmentTable2025-03-04.csv
    4. HITTable2025-03-04.csv

but it wasn't present in any of them

though, It is present in the filded_CSN data

## Did this person get paid?
It is not present in the final payments data but is he paid based on the all_apps_wide data?

- participant.payoff column is the final payment (base + bonus) to responders

In [65]:
msk = df_apps[c.mturk_id].eq(list(sym_diff)[0])
print(df_apps.loc[msk , 'participant.payoff'])

105    1.6
Name: participant.payoff, dtype: float64


### Yes
got paid $1.6 base payment without any bonus so the final payments data is apparently not complete as well as other payments data files

##  I KEEP THIS ONE RESPONDER AS A VALID RESPONSE
So I assume this is a valid response and keep it

# Check on whether the set of paid ones all have MTurk IDs, in the apps_wide data

In [66]:
df_1 = df_apps_wide__raw.dropna(subset = [c.mturk_id])

msk = df_apps_wide__raw[c.participant_payoff].eq(0)
msk = msk | df_apps_wide__raw[c.participant_payoff].isna()

df_2 = df_apps_wide__raw[~ msk]

df_1.eq(df_2).all(axis = None)

np.False_

## ISSUE: There are 4 responders with no MTurk ID but they have a participant.payoff value

# NEXT CHECK: Are all those who have participant.payoff have non-zero participant.payoff?
In case if someone got bonus but the total payment is zero, then it is an issue

In [67]:
msk = df_apps_wide__raw[c.participant_bonus].notna()
df_1 = df_apps_wide__raw[msk]

check = df_1[c.participant_payoff].ne(0).all()

print(f"All responders with participant bonus have non-zero participant payoff: {check}")

All responders with participant bonus have non-zero participant payoff: True


No Issue, all responders with participant bonus have non-zero participant payoff

# Next check: is the final payments the same as the all_apps_wide data? except for that one inconsistency?

In [69]:
cols_2_keep = [c.mturk_id , c.participant_payoff , c.participant_bonus]

df_1 = df_apps_wide__raw[cols_2_keep]

df_1 = df_1.dropna(subset = [c.mturk_id])
df_1.shape

(47, 3)

In [70]:
df_2 = df_payments__raw.copy()
df_2 = df_2.dropna(subset = [c.workerId])
df_2 = df_2.rename(columns = {
        c.workerId : c.mturk_id
        })

df_2.shape

(46, 5)

In [74]:
df_merged = pd.merge(df1 , df_2)

df_merged = df_merged.convert_dtypes()

df_merged = df_merged.fillna(0)
df_merged = df_merged.round(1)  # round to 2 decimal places because other wise the equla numbers are not equal exactly because of floating point precision issues

# is paticipant.payoff (from all_apps_wide) == total (from Final Payments.xlsx)
df_merged[c.test] = df_merged[c.participant_payoff].eq(df_merged[c.total])
print(f"All <{c.participant_payoff}> values are equal to <{c.total}> in the Final Payments.xlsx:" ,
      df_merged[c.test].all())

# is participant.bonus (from all_apps_wide) == bonus (from Final Payments.xlsx)
df_merged[c.test] = df_merged[c.participant_bonus].eq(df_merged[c.bonus])
print(f"All <{c.participant_bonus} values are equal to <{c.bonus}> the Final Payments.xlsx:" ,
      df_merged[c.test].all())

All <participant.payoff> values are equal to <Total> in the Final Payments.xlsx: True
All <participant.bonus values are equal to <Bonus> the Final Payments.xlsx: True


# Keep Valid Responses
CRITERIA:
- I used all who paid (participant.payoff > 0) from the apps_wide data as valid responses
- This is a greater set than the one from the Final Payments.xlsx file and the ones who have MTurk IDs
- I just wanted to use the largest set of valid responses
- But we need to check whether what has happened with the ones who have no MTurk IDs

In [80]:
msk = df_apps_wide__raw[c.participant_payoff].gt(0)

df_all_apps_cln = df_apps_wide__raw[msk].copy()
df_all_apps_cln.shape

In [82]:
# check all the participant.code values are unique
df_all_apps_cln[c.participant_code].is_unique

True

## Yes all participant.code values are unique in the cleaned data

In [91]:
# save the cleaned data
df_all_apps_cln.to_excel(fp.clean_all_apps , index = False)

In [92]:
# keep the final sample of responders from the page times data as clean page times data

df_pg_times = pd.read_csv(fp.page_times)
df_pg_times.shape

(946, 10)

In [97]:
# rename participant_code to participant.code in the page times data to match the all_apps data
df_pg_times = df_pg_times.rename(columns = {
        'participant_code' : c.participant_code
        })

In [98]:
msk = df_pg_times[c.participant_code].isin(df_all_apps_cln[c.participant_code])

df_pg_times_cln = df_pg_times[msk].copy()

df_pg_times_cln.shape

(943, 10)

In [100]:
# sort on participant.code and page index
df_pg_times_cln = df_pg_times_cln.sort_values(by = [c.participant_code ,
                                                    c.page_index])

In [104]:
# save the cleaned page times data
df_pg_times_cln.to_excel(fp.cln_page_times , index = False)